# Experimental Results: Histogram

This notebook is designed for visualizing and comparing the histograms of color channels (Red, Green, and Blue) across three sets of images: input images, reference images, and output images. The tool loads images from three different folders, calculates their histograms, and then provides an interactive visualization of these histograms. The histograms are displayed separately for each color channel and also combined in a single plot for easy comparison.

### Import Dependencies

In [ ]:
!pip install ipywidgets

import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
from ipywidgets import interact, fixed

### How it Works (Briefly)
Loading Images and Histograms: The load_images_and_histograms function reads images from three provided folders (input, reference, and output). It matches images based on their filenames, loads the corresponding images, and computes their histograms for the Red, Green, and Blue channels using OpenCV's cv2.calcHist() method.

Interactive Visualization: The interactive_image_histogram function enables interaction by using the interact function from the ipywidgets library. It allows the user to select a specific image comparison (input, reference, and output) from the available dataset.

Plotting Histograms: The plot_image_histogram_comparison function creates a plot displaying the images and their respective histograms. It arranges the images and their histograms in a 5x3 grid, where:

The first row shows the images (input, reference, and output).
The second, third, and fourth rows display the individual histograms for each color channel (Red, Green, and Blue) for input, reference, and output images.
The fifth row shows the combined (overlapping) histograms for all color channels in each image.

In [ ]:

def load_images_and_histograms(input_folder, reference_folder, output_folder):
    comparisons = {}
    
    for file in os.listdir(output_folder):
        if not file.lower().endswith(('.png', '.jpg', '.jpeg')):
            continue
        
        parts = file.split("_")
        if len(parts) < 2:
            print(f"Skipping file with unexpected format: {file}")
            continue
        
        input_name = parts[0]
        reference_name = parts[1].rsplit(".", 1)[0]
        
        input_path = os.path.join(input_folder, f"{input_name}.jpg")
        reference_path = os.path.join(reference_folder, f"{reference_name}.jpg")
        output_path = os.path.join(output_folder, file)
        
        if not (os.path.exists(input_path) and os.path.exists(reference_path) and os.path.exists(output_path)):
            print(f"Missing one of the images for {file}")
            continue
        
        input_img = cv2.imread(input_path)
        reference_img = cv2.imread(reference_path)
        output_img = cv2.imread(output_path)
        
        if input_img is None or reference_img is None or output_img is None:
            print(f"Failed to load one of the images for {file}")
            continue
        
        histograms = {
            "input": [cv2.calcHist([input_img], [i], None, [256], [0, 256]) for i in range(3)],
            "reference": [cv2.calcHist([reference_img], [i], None, [256], [0, 256]) for i in range(3)],
            "output": [cv2.calcHist([output_img], [i], None, [256], [0, 256]) for i in range(3)],
        }
        
        comparisons[file] = {
            "input_img": input_img,
            "reference_img": reference_img,
            "output_img": output_img,
            "histograms": histograms
        }
    
    return comparisons

def plot_image_histogram_comparison(data, file_name):
    comparison = data[file_name]
    input_img = comparison["input_img"]
    reference_img = comparison["reference_img"]
    output_img = comparison["output_img"]
    histograms = comparison["histograms"]
    
    # Find the maximum histogram value to set consistent y-axis limits
    max_hist_value = max(np.max(histograms["input"][i]) for i in range(3))
    max_hist_value = max(max_hist_value, max(np.max(histograms["reference"][i]) for i in range(3)))
    max_hist_value = max(max_hist_value, max(np.max(histograms["output"][i]) for i in range(3)))
    
    fig, axes = plt.subplots(5, 3, figsize=(20, 25))  # Increase the number of rows to 5
    
    # Display input, reference, and output images
    axes[0, 0].imshow(cv2.cvtColor(input_img, cv2.COLOR_BGR2RGB))
    axes[0, 0].set_title("Input Image")
    axes[0, 0].axis("off")
    
    axes[0, 1].imshow(cv2.cvtColor(reference_img, cv2.COLOR_BGR2RGB))
    axes[0, 1].set_title("Reference Image")
    axes[0, 1].axis("off")
    
    axes[0, 2].imshow(cv2.cvtColor(output_img, cv2.COLOR_BGR2RGB))
    axes[0, 2].set_title("Output Image")
    axes[0, 2].axis("off")
    
    # Plot histograms for each color channel (input, reference, and output)
    colors = ['b', 'g', 'r']
    
    # Input Histograms (Row 1, Columns 0-2)
    for i, color in enumerate(colors):
        axes[1, i].plot(histograms["input"][i], color=color)
        axes[1, i].set_title(f"Input Histogram ({color.upper()})")
        axes[1, i].set_xlim(0, 256)
        axes[1, i].set_ylim(0, max_hist_value)  # Same y-axis limit for all histograms
        
    # Reference Histograms (Row 2, Columns 0-2)
    for i, color in enumerate(colors):
        axes[2, i].plot(histograms["reference"][i], color=color)
        axes[2, i].set_title(f"Reference Histogram ({color.upper()})")
        axes[2, i].set_xlim(0, 256)
        axes[2, i].set_ylim(0, max_hist_value)  # Same y-axis limit for all histograms
        
    # Output Histograms (Row 3, Columns 0-2)
    for i, color in enumerate(colors):
        axes[3, i].plot(histograms["output"][i], color=color)
        axes[3, i].set_title(f"Output Histogram ({color.upper()})")
        axes[3, i].set_xlim(0, 256)
        axes[3, i].set_ylim(0, max_hist_value)  # Same y-axis limit for all histograms
    
    # Combined RGB Histograms (Row 4, Columns 0-2) - Overlapping RGB histograms for each image
    axes[4, 0].plot(histograms["input"][0], color='b', alpha=0.5, label="Blue")
    axes[4, 0].plot(histograms["input"][1], color='g', alpha=0.5, label="Green")
    axes[4, 0].plot(histograms["input"][2], color='r', alpha=0.5, label="Red")
    axes[4, 0].set_title("Combined Input Histogram")
    axes[4, 0].set_xlim(0, 256)
    axes[4, 0].set_ylim(0, max_hist_value)
    axes[4, 0].legend()
    
    axes[4, 1].plot(histograms["reference"][0], color='b', alpha=0.5, label="Blue")
    axes[4, 1].plot(histograms["reference"][1], color='g', alpha=0.5, label="Green")
    axes[4, 1].plot(histograms["reference"][2], color='r', alpha=0.5, label="Red")
    axes[4, 1].set_title("Combined Reference Histogram")
    axes[4, 1].set_xlim(0, 256)
    axes[4, 1].set_ylim(0, max_hist_value)
    axes[4, 1].legend()
    
    axes[4, 2].plot(histograms["output"][0], color='b', alpha=0.5, label="Blue")
    axes[4, 2].plot(histograms["output"][1], color='g', alpha=0.5, label="Green")
    axes[4, 2].plot(histograms["output"][2], color='r', alpha=0.5, label="Red")
    axes[4, 2].set_title("Combined Output Histogram")
    axes[4, 2].set_xlim(0, 256)
    axes[4, 2].set_ylim(0, max_hist_value)
    axes[4, 2].legend()

    # Hide the axis of the extra rows (to keep the grid layout clean)
    for row in range(5):
        for col in range(3):
            if row == 0 and col > 2:  # Remove image titles for unnecessary columns
                axes[row, col].axis("off")
            if row == 4 and col > 2:  # Remove combined histograms for unnecessary columns
                axes[row, col].axis("off")
    
    plt.tight_layout()
    plt.show()


def interactive_image_histogram(input_folder, reference_folder, output_folder):
    data = load_images_and_histograms(input_folder, reference_folder, output_folder)
    
    if not data:
        print("No valid comparisons found.")
        return
    
    interact(plot_image_histogram_comparison, data=fixed(data), file_name=list(data.keys()))


Running the comparison codes

In [60]:
current_dir = os.getcwd()
input_folder = os.path.join(current_dir, 'input')
reference_folder = os.path.join(current_dir, 'reference')
output_folder = os.path.join(current_dir, 'output')

interactive_image_histogram(input_folder, reference_folder, output_folder)


Missing one of the images for day-mountains_main-input.png
Missing one of the images for green-forest_main-input.png
Missing one of the images for green-land_main-input.png
Missing one of the images for main-input_day-mountains.png
Missing one of the images for main-input_green-forest.png
Missing one of the images for main-input_green-land.png
Missing one of the images for main-input_main-reference.png
Missing one of the images for main-input_pink-sunset.png
Missing one of the images for main-input_purple-field.png
Missing one of the images for main-input_yellow-sunset.png
Missing one of the images for main-reference_main-input.png
Missing one of the images for pink-sunset_main-input.png
Missing one of the images for purple-field_main-input.png
Missing one of the images for yellow-sunset_main-input.png


interactive(children=(Dropdown(description='file_name', options=('day-mountains_green-forest.png', 'day-mounta…

In [50]:
from ipywidgets import interact, fixed
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

def load_images_and_histograms(input_folder, reference_folder, output_folder):
    comparisons = {}
    
    for file in os.listdir(output_folder):
        if not file.lower().endswith(('.png', '.jpg', '.jpeg')):
            continue
        
        parts = file.split("_")
        if len(parts) < 2:
            print(f"Skipping file with unexpected format: {file}")
            continue
        
        input_name = parts[0]
        reference_name = parts[1].rsplit(".", 1)[0]
        
        input_path = os.path.join(input_folder, f"{input_name}.jpg")
        reference_path = os.path.join(reference_folder, f"{reference_name}.jpg")
        output_path = os.path.join(output_folder, file)
        
        if not (os.path.exists(input_path) and os.path.exists(reference_path) and os.path.exists(output_path)):
            print(f"Missing one of the images for {file}")
            continue
        
        input_img = cv2.imread(input_path)
        reference_img = cv2.imread(reference_path)
        output_img = cv2.imread(output_path)
        
        if input_img is None or reference_img is None or output_img is None:
            print(f"Failed to load one of the images for {file}")
            continue
        
        # Calculate RGB histograms
        histograms = {
            "input": [cv2.calcHist([input_img], [i], None, [256], [0, 256]) for i in range(3)],
            "reference": [cv2.calcHist([reference_img], [i], None, [256], [0, 256]) for i in range(3)],
            "output": [cv2.calcHist([output_img], [i], None, [256], [0, 256]) for i in range(3)],
        }
        
        # Convert images to LAB color space
        lab_input = cv2.cvtColor(input_img, cv2.COLOR_BGR2LAB)
        lab_reference = cv2.cvtColor(reference_img, cv2.COLOR_BGR2LAB)
        lab_output = cv2.cvtColor(output_img, cv2.COLOR_BGR2LAB)
        
        # Split LAB channels
        lab_channels = {
            "input": cv2.split(lab_input),
            "reference": cv2.split(lab_reference),
            "output": cv2.split(lab_output)
        }
        
        comparisons[file] = {
            "input_img": input_img,
            "reference_img": reference_img,
            "output_img": output_img,
            "histograms": histograms,
            "lab_channels": lab_channels
        }
    
    return comparisons

def plot_image_histogram_and_lab_comparison(data, file_name):
    comparison = data[file_name]
    input_img = comparison["input_img"]
    reference_img = comparison["reference_img"]
    output_img = comparison["output_img"]
    histograms = comparison["histograms"]
    lab_channels = comparison["lab_channels"]
    
    # Find the maximum histogram value to set consistent y-axis limits
    max_hist_value = max(np.max(histograms["input"][i]) for i in range(3))
    max_hist_value = max(max_hist_value, max(np.max(histograms["reference"][i]) for i in range(3)))
    max_hist_value = max(max_hist_value, max(np.max(histograms["output"][i]) for i in range(3)))
    
    fig, axes = plt.subplots(5, 3, figsize=(20, 25))
    
    # Display input, reference, and output images
    axes[0, 0].imshow(cv2.cvtColor(input_img, cv2.COLOR_BGR2RGB))
    axes[0, 0].set_title("Input Image")
    axes[0, 0].axis("off")
    
    axes[0, 1].imshow(cv2.cvtColor(reference_img, cv2.COLOR_BGR2RGB))
    axes[0, 1].set_title("Reference Image")
    axes[0, 1].axis("off")
    
    axes[0, 2].imshow(cv2.cvtColor(output_img, cv2.COLOR_BGR2RGB))
    axes[0, 2].set_title("Output Image")
    axes[0, 2].axis("off")
    
    # Plot histograms for each color channel (input, reference, and output)
    colors = ['b', 'g', 'r']
    
    # Input Histograms (Row 1, Columns 0-2)
    for i, color in enumerate(colors):
        axes[1, i].plot(histograms["input"][i], color=color)
        axes[1, i].set_title(f"Input Histogram ({color.upper()})")
        axes[1, i].set_xlim(0, 256)
        axes[1, i].set_ylim(0, max_hist_value)  # Same y-axis limit for all histograms
        
    # Reference Histograms (Row 2, Columns 0-2)
    for i, color in enumerate(colors):
        axes[2, i].plot(histograms["reference"][i], color=color)
        axes[2, i].set_title(f"Reference Histogram ({color.upper()})")
        axes[2, i].set_xlim(0, 256)
        axes[2, i].set_ylim(0, max_hist_value)  # Same y-axis limit for all histograms
        
    # Output Histograms (Row 3, Columns 0-2)
    for i, color in enumerate(colors):
        axes[3, i].plot(histograms["output"][i], color=color)
        axes[3, i].set_title(f"Output Histogram ({color.upper()})")
        axes[3, i].set_xlim(0, 256)
        axes[3, i].set_ylim(0, max_hist_value)  # Same y-axis limit for all histograms
    
    # Plot LAB channels (L, A, B)
    # Input LAB Channels (Row 4, Columns 0-2)
    lab_titles = ['L Channel', 'A Channel', 'B Channel']
    for i, channel in enumerate(lab_channels["input"]):
        axes[4, i].imshow(channel, cmap='gray')
        axes[4, i].set_title(f"Input {lab_titles[i]}")
        axes[4, i].axis('off')
    
    # Hide the axis of the extra rows (to keep the grid layout clean)
    for row in range(5):
        for col in range(3):
            if row == 0 and col > 2:  # Remove image titles for unnecessary columns
                axes[row, col].axis("off")
            if row == 4 and col > 2:  # Remove LAB channels for unnecessary columns
                axes[row, col].axis("off")
    
    plt.tight_layout()
    plt.show()

def interactive_image_histogram_and_lab(input_folder, reference_folder, output_folder):
    data = load_images_and_histograms(input_folder, reference_folder, output_folder)
    
    if not data:
        print("No valid comparisons found.")
        return
    
    interact(plot_image_histogram_and_lab_comparison, data=fixed(data), file_name=list(data.keys()))


In [51]:
current_dir = os.getcwd()
input_folder = os.path.join(current_dir, 'input')
reference_folder = os.path.join(current_dir, 'reference')
output_folder = os.path.join(current_dir, 'output')

interactive_image_histogram(input_folder, reference_folder, output_folder)


Missing one of the images for day-mountains_main-input.png
Missing one of the images for green-forest_main-input.png
Missing one of the images for green-land_main-input.png
Missing one of the images for main-input_day-mountains.png
Missing one of the images for main-input_green-forest.png
Missing one of the images for main-input_green-land.png
Missing one of the images for main-input_main-reference.png
Missing one of the images for main-input_pink-sunset.png
Missing one of the images for main-input_purple-field.png
Missing one of the images for main-input_yellow-sunset.png
Missing one of the images for main-reference_main-input.png
Missing one of the images for pink-sunset_main-input.png
Missing one of the images for purple-field_main-input.png
Missing one of the images for yellow-sunset_main-input.png


interactive(children=(Dropdown(description='file_name', options=('day-mountains_green-forest.png', 'day-mounta…